In [4]:
import numpy as np
import time
import plotly.graph_objects as go

In [5]:
# -------------------- 2-D Ackley Function --------------------
def ackley_2d(xy):
    """
    2D Ackley (for plotting + DE):
      f(x,y) = -20 * exp(-0.2 * sqrt(0.5*(x^2 + y^2)))
               - exp(0.5*(cos(2πx) + cos(2πy)))
               + 20 + e
    Input: xy is an array-like of length 2.
    """
    x, y = xy[0], xy[1]
    a = 20.0
    b = 0.2
    c = 2.0 * np.pi

    sum_sq = x**2 + y**2
    term1 = -a * np.exp(-b * np.sqrt(0.5 * sum_sq))
    term2 = -np.exp(0.5 * (np.cos(c * x) + np.cos(c * y)))
    return term1 + term2 + a + np.e

In [6]:
# -------------------- DE Parameters (2D case) --------------------
D = 2                    # NOW 2 dimensions (x,y)
lower_bound = -5.0       # we’ll plot over [-5,5] for clarity
upper_bound =  5.0

NP = 20                 # population size
F  = 0.85               # mutation factor
CR = 0.8                # crossover probability
max_gen = 200           # max generations
tol    = 1e-6           # stop if best fitness ≤ tol


In [7]:
# -------------------- Initialize Population --------------------
np.random.seed(42)
pop = np.random.uniform(low=lower_bound, high=upper_bound, size=(NP, D))
fitness = np.array([ackley_2d(ind) for ind in pop])

best_idx     = np.argmin(fitness)
best_vec     = pop[best_idx].copy()
best_fit     = fitness[best_idx]

# Lists to store the “best” path
best_path_xy = [best_vec.copy()]   # store (x,y)
best_path_z  = [best_fit]          # store f(x,y)

print(f"Gen 0: Best f = {best_fit:.6f}, Best (x,y) = {best_vec}")

Gen 0: Best f = 5.749673, Best (x,y) = [1.01115012 2.08072578]


In [8]:
# -------------------- DE Main Loop (2D) --------------------
for gen in range(1, max_gen+1):
    for i in range(NP):
        # 1) Mutation (DE/rand/1)
        indices = list(range(NP))
        indices.remove(i)
        r1, r2, r3 = np.random.choice(indices, 3, replace=False)
        x_r1, x_r2, x_r3 = pop[r1], pop[r2], pop[r3]

        donor = x_r1 + F * (x_r2 - x_r3)
        donor = np.clip(donor, lower_bound, upper_bound)

        # 2) Binomial Crossover
        trial = np.empty(D)
        j_rand = np.random.randint(D)
        for j in range(D):
            if np.random.rand() < CR or j == j_rand:
                trial[j] = donor[j]
            else:
                trial[j] = pop[i,j]

        # 3) Selection
        f_trial = ackley_2d(trial)
        if f_trial <= fitness[i]:
            pop[i] = trial
            fitness[i] = f_trial
            if f_trial < best_fit:
                best_fit = f_trial
                best_vec = trial.copy()

    best_path_xy.append(best_vec.copy())
    best_path_z.append(best_fit)
    if gen % 10 == 0 or best_fit <= tol:
        print(f"Gen {gen:3d}: Best f = {best_fit:.6f}, Best (x,y) = {best_vec}")

    if best_fit <= tol:
        print(f"\n→ Early stop at Gen {gen} (f ≤ {tol}).")
        break

print(f"\nFinal: Best f = {best_fit:.6f}, Best (x,y) = {best_vec}")

Gen  10: Best f = 1.103010, Best (x,y) = [ 0.03119941 -0.16757661]
Gen  20: Best f = 0.161685, Best (x,y) = [-0.03215115 -0.02591185]
Gen  30: Best f = 0.018479, Best (x,y) = [-0.00616323  0.00037176]
Gen  40: Best f = 0.003397, Best (x,y) = [0.00025877 0.00115929]
Gen  50: Best f = 0.000621, Best (x,y) = [-0.00016938  0.00013888]
Gen  60: Best f = 0.000073, Best (x,y) = [ 1.25981176e-05 -2.24681618e-05]
Gen  70: Best f = 0.000013, Best (x,y) = [-2.59657564e-06  3.75115029e-06]
Gen  78: Best f = 0.000000, Best (x,y) = [ 7.42384769e-08 -8.69888304e-08]

→ Early stop at Gen 78 (f ≤ 1e-06).

Final: Best f = 0.000000, Best (x,y) = [ 7.42384769e-08 -8.69888304e-08]


In [9]:
# -------------------- Build 3D Surface of Ackley over [-5,5]^2 --------------------
grid_n = 200
xs = np.linspace(lower_bound, upper_bound, grid_n)
ys = np.linspace(lower_bound, upper_bound, grid_n)
X, Y = np.meshgrid(xs, ys)
Z = -20 * np.exp(-0.2 * np.sqrt((X**2 + Y**2) / 2)) \
    - np.exp(0.5 * (np.cos(2*np.pi*X) + np.cos(2*np.pi*Y))) \
    + 20 + np.e

In [10]:
# -------------------- Plotly 3D--------------------
fig = go.Figure()

# 1) Ackley surface
fig.add_trace(go.Surface(
    x=X, y=Y, z=Z,
    colorscale="Viridis",
    opacity=0.8,
    name="Ackley Surface"
))

# 2) DE best‐path as a 3D line + markers
best_path_xy = np.array(best_path_xy)  # shape (gens+1, 2)
best_path_z  = np.array(best_path_z)   # shape (gens+1,)

fig.add_trace(go.Scatter3d(
    x=best_path_xy[:,0],         # x coordinates
    y=best_path_xy[:,1],         # y coordinates
    z=best_path_z,               # f(x,y)
    mode="lines+markers",
    marker=dict(size=3, color="red"),
    line=dict(color="red", width=2),
    name="DE Best Path"
))

fig.update_layout(
    title="DE Trajectory over 3D Ackley Surface (D=2)",
    scene=dict(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="f(x,y)",
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.